In [1]:
# import needed dependencies

#import yolo 
from ultralytics import YOLO

# import torch 
import torch
from torchvision import transforms

# import os and PIL for model evaluation 
import os
from PIL import Image

# import cv2 for image handling
import cv2


#import numpy
import numpy as np

#importing mqtt dependencies
import paho.mqtt.client as mqtt
import time


#chechking torch version
print(torch.__version__)

#setting manual seed
torch.manual_seed(42)

#clearing out the allocated storage used by the previous model
torch.cuda.empty_cache()



2.1.1+cu121


In [2]:
model_path = os.path.join('.', 'runs', 'detect', 'train6', 'weights', 'best.pt')
model = YOLO(model_path)

In [3]:
# create a client to send data to the plc
def start_connection_with_mqtt(client_id, MqttBroker):
    client  = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1,client_id)
    client.connect(MqttBroker)
    return client

def publish_message(client, message, topic):
    client.publish(topic, message)
    print(f"Published: {message} to topic {topic}")
    #time.sleep(0.1)
node_red_client = start_connection_with_mqtt("bottle_detectetion", "broker.hivemq.com")

C:\Users\omarm\AppData\Local\Temp\ipykernel_14744\3866503338.py:3: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client  = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1,client_id)


In [4]:
# to be compeleted
#def send_plc_signal(client, class_id, topic): #old arguments --> frame, predictions

#    # Get image dimensions
#     width = frame.shape[1]
#     # Draw bounding boxes and labels on the image
#     for pred in predictions:

#         # Extracxting needed data from model predictions
#         class_id = pred.boxes.data.tolist()[0][5]
#         x_center = pred.boxes.xywhn.tolist()[0][0]

        
#         #checking weither the center point crosses the center of the camera frame or not 
#         if x_center * width >= (width / 2) - 5 and x_center * width <= (width / 2) + 5 :
#             # send signal to plc
   # publish_message(client, class_id, topic)
    #time.sleep(1)

In [5]:
def send_plc_signal(client, class_id, topic):
    publish_message(client, class_id, topic)

In [6]:
def visualize_predictions(frame, predictions, half_frame_up, half_frame_down):

   # Get image dimensions
    height, width, _ = frame.shape
    # Draw bounding boxes and labels on the image
    for pred in predictions:

        # Extracxting needed data from model predictions
        class_id, confidence = pred.boxes.data.tolist()[0][5], pred.boxes.data.tolist()[0][4]
        class_name_dict = pred.names
        x_center, y_center, bbox_width, bbox_height = pred.boxes.xywhn.tolist()[0]
        if confidence > 0.7:
        # some visualization to see what is happening
            down = (int(x_center * width), 0)
            up = (int(x_center * width), height)
            cv2.line(frame, pt1 = down, pt2 = up, color = (255, 255, 255), thickness=2)
            
            # Convert relative coordinates to absolute coordinates
            x1 = int((x_center - bbox_width / 2) * width)
            y1 = int((y_center - bbox_height / 2) * height)
            x2 = int((x_center + bbox_width / 2) * width)
            y2 = int((y_center + bbox_height / 2) * height)
            
            #checking weither the center point crosses the center of the camera frame or not 
            #if x_center * width == (width / 2):
            if x_center * width >= (width / 2) - 2 and x_center * width <= (width / 2) + 2 :
                cv2.line(img = frame, pt1= half_frame_up, pt2 = half_frame_down, color = (0, 0, 0), thickness = 3)
                # send 1 to the node-red and wait for acknowladgement then send the class id
                send_plc_signal(node_red_client, 1, "bottle_detected")
                send_plc_signal(node_red_client, class_id + 2, "bottle_color")
            

            # Draw bounding box
            cv2.rectangle(img = frame, pt1 = (x1, y1), pt2 = (x2, y2), color = (0, 255, 0), thickness = 2)
            
            # Write class label and confidence
            label = f'{class_name_dict[class_id]}: accu :{confidence:.2f}'
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2) 

        return frame

# This Function is done with out that much calculation for the raspberry pi


In [7]:
half_frame_down = (320, 0)

half_frame_up = (320,480)

color1= (0, 255, 0)

color2 = (255, 0, 0)

In [10]:
cap = cv2.VideoCapture(0)

while True :

    ret, frame = cap.read()

    if not ret :
        break

    # doing forward path for the model to predict our bounding boxes
    with torch.inference_mode():
        results = model(frame)

    # visualizting predictions
    if len(results[0].boxes.data) != 0 :
        frame = visualize_predictions(frame, results, half_frame_up, half_frame_down)
    else :
        send_plc_signal(node_red_client, 0, "bottle_detected")
        send_plc_signal(node_red_client, 0, "bottle_color")
        
        # #send signal to plc to stop convyer
        # send_plc_signal(frame)
        
    # opening the web cam with predictions
    cv2.imshow("predictions", frame)

    # if you press s then break the programm
    if cv2.waitKey(1) & 0xFF == ord('s'):
        break

# release all allocated memory 
cap.release()

# destroying the windows
cv2.destroyAllWindows() 


0: 480x640 1 blue bottle, 11.9ms
Speed: 2.0ms preprocess, 11.9ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 blue bottle, 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 blue bottle, 10.7ms
Speed: 1.4ms preprocess, 10.7ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 blue bottle, 13.2ms
Speed: 1.5ms preprocess, 13.2ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 blue bottle, 12.8ms
Speed: 2.2ms preprocess, 12.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 blue bottle, 12.3ms
Speed: 1.2ms preprocess, 12.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 blue bottles, 10.7ms
Speed: 1.3ms preprocess, 10.7ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0